In [3]:
# !pip uninstall tflearn
!pip install git+https://github.com/MihaMarkic/tflearn.git@fix/is_sequence_missing -q

  Preparing metadata (setup.py) ... done


In [4]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
from tensorflow.python.framework import ops
ops.reset_default_graph()
import random


Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
import json
with open('intents.json') as file:
    data = json.load(file)

In [7]:
words = []
labels = []
docs_x = []
docs_y = []

In [8]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [9]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [10]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [11]:
training = numpy.array(training)
output = numpy.array(output)

In [12]:
len(output[0])

13

In [13]:
ops.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(training, output, n_epoch=1020, batch_size=8, show_metric=True)
model.save("/content/modelpaing.tflearn")

Training Step: 17199  | total loss: 0.07491 | time: 0.040s
| Adam | epoch: 1720 | loss: 0.07491 - acc: 0.9808 -- iter: 72/74
Training Step: 17200  | total loss: 0.06755 | time: 0.044s
| Adam | epoch: 1720 | loss: 0.06755 - acc: 0.9827 -- iter: 74/74
--


In [ ]:
# try:
#     model.load("/content/model.tflearn")
# except:
#     model.fit(training, output, n_epoch=100, batch_size=8, show_metric=True)
#     model.save("/content/model.tflearn")

In [14]:
model.load("/content/modelpaing.tflearn")

In [15]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: Bisakah saya membayar dengan kartu kredit?
Anda dapat membayar dengan berbagai metode pembayaran, seperti kartu kredit, kartu debit, uang tunai, dan transfer bank.
You: Bisakah saya memberikan saran atau masukan?
Jika Anda tidak puas dengan produk atau layanan kami, Anda dapat memberikan komplain melalui email, telepon, atau media sosial. Kami akan berusaha untuk menyelesaikan masalah Anda dengan cepat dan tepat.
You: Apa saja produk yang sedang diskon?
Ya, saat ini kami sedang ada diskon untuk produk susu tertentu. Silakan kunjungi website kami untuk melihat informasi lebih lanjut.
You: see you later
Sampai jumpa lagi! Senang bisa membantu Anda.
You: Bisakah saya pesan online?
Ya, Anda dapat memesan produk susu secara online melalui website kami.
You: Apakah Anda menggunakan bahan-bahan organik?
Ya, semua produk kami diproduksi dengan standar keamanan yang tinggi. Kami memastikan bahwa semua produk kami aman untuk dikonsumsi.
You: B